In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
from jax import numpy as jnp
from jax.experimental import host_callback

from tqdm.auto import tqdm
from functools import partial
import optax
import matplotlib.pyplot as plt
from pprint import pprint

from flax import linen as nn
from typing import Sequence

from functools import partial
from typing import Callable, Sequence
from flax import struct
from flax import linen as nn
import jax
import jax.numpy as jnp
import optax
from functools import partial
import numpy as np

import actions
import qlearning
import replay
import plotting
from utils import ConvNet, Transition
from frozen_lake import EnvState, FrozenLake, ObsType, ActType, RNGKey


In [7]:
jax.devices()

[cuda(id=0)]

In [8]:
import frozen_lake, utils, replay, qlearning
rng_key = jax.random.PRNGKey(0)
map_size = 8
rng_env_init, rng_env_reset, rng_dql = jax.random.split(rng_key, 3)
env = frozen_lake.FrozenLake.make(rng_env_init, (map_size, map_size))
env_state, env_obs = env.reset(rng_env_reset)

In [9]:
@partial(jax.jit, static_argnames=("steps"))
def random_rollout(
    env: FrozenLake, rng_key: RNGKey, steps: int
):

    def scan_compatible_step(carry, rng_key: RNGKey):
        env_state, obs = carry
        rng_action, rng_step, rng_reset = jax.random.split(rng_key, 3)
        action = jax.random.randint(rng_action, (), 0, 4)
        next_env_state, next_obs, reward, done, info = env.step(env_state, rng_step, action)
        transition = Transition(env_state, obs, action, next_obs, reward, done, info)

        # reset the environment if done
        next_env_state, next_obs = jax.lax.cond(
            done,
            lambda: env.reset(rng_reset),
            lambda: (next_env_state, next_obs),
        )
        return (next_env_state, next_obs), transition

    rng_env_reset, rng_steps = jax.random.split(rng_key)
    rng_steps = jax.random.split(rng_steps, steps)
    env_state, obs = env.reset(rng_env_reset)
    _, transitions = jax.lax.scan(scan_compatible_step, (env_state, obs), rng_steps)
    return transitions

In [11]:
%timeit random_rollout(env, rng_key, 256)

27.1 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


: 